# Recommendation System

In [ ]:
import numpy as np 
import pandas as pd 

### Importing Datasets

In [ ]:
movies=pd.read_csv('datasets/tmdb_5000_movies.csv')
credits=pd.read_csv('datasets/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies.shape

In [ ]:
credits.shape

### Data Joining

In [ ]:
df=movies.merge(credits,on='title')

In [ ]:
df.shape

In [ ]:
df.head(1)

### Columns Filtering 

In [ ]:
df=df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
df.info()

In [ ]:
df.iloc[0].genres

### Data Preprocessing

In [ ]:
import ast
l2=[]
def convert(l1):
    for i in ast.literal_eval(l1):
        l2.append(i['name'])
    return l2

In [ ]:
df['genres']=df['genres'].apply(convert)

In [ ]:
df['keywords']=df['keywords'].apply(convert)

In [ ]:
df.iloc[0].cast

In [ ]:
def convert3(l1):
    l2=[]
    counter=0
    for i in ast.literal_eval(l1):
        if counter!=3:
            l2.append(i['name'])
            counter+=1
        else:
            break
    return l2

In [ ]:
df['cast']=df['cast'].apply(convert3)

In [ ]:
def convert2(l1):
    l2=[]
    for i in ast.literal_eval(l1):
        if i['job']=='Director':
            l2.append(i['name'])
            break
    return l2
   

In [ ]:
df['crew']=df['crew'].apply(convert2)

In [ ]:
df.head(1)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [ ]:
df.head(1)

In [ ]:
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ","") for i in x])


In [ ]:
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ","") for i in x])


In [ ]:
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ","") for i in x])


In [ ]:
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
df['tags']=df['overview']+df['genres']+df['keywords']+df['cast']+df['crew']

### Final filtering of columns

In [ ]:
new_df=df[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join (x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

### Stemming the tags column

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df.head()

### Vectorization of data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

### Cosine Similarity

In [ ]:
from sklearn. metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity.shape

In [ ]:
def recommend (movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')

### Generating Pickle Files

In [ ]:
import pickle

In [ ]:
pickle.dump (new_df.to_dict() ,open('movie_dict.pk1', 'wb' ))

In [ ]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))